In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv("../../../Downloads/movies.csv")
ratings = pd.read_csv("../../../Downloads/ratings.csv")

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# Dropping timestamp column (unused)
ratings.drop('timestamp', axis=1, inplace=True)
ratings.head()

,userId,movieId,rating
0,1,16,4.0
1,1,24,1.5
2,1,32,4.0
3,1,47,4.0
4,1,50,4.0


# Content-Based Filtering
<hr>

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
convec = CountVectorizer()

genvec = convec.fit_transform(movies['genres'])
simScore = cosine_similarity(genvec.toarray())

In [137]:
# Input to search movie by title
search = input('Insert your movie title here: ').lower()
searchedMovies = movies.loc[movies['title'].str.lower().str.contains(search)].index[0]
movies.iloc[searchedMovies]

Insert your movie title here:  superman vs.


movieId                            94974
title      Superman vs. The Elite (2012)
genres                  Action|Animation
Name: 9370, dtype: object

In [149]:
similarMovies = list(enumerate(simScore[searchedMovies]))
similarMovies = sorted(similarMovies, key=lambda x: x[1], reverse=True)

dfRecommendedMovies = []
for i in similarMovies[:6]:
    dfRecommendedMovies.append(movies.iloc[i[0]])
dfRecommendedMovies = pd.DataFrame(dfRecommendedMovies)

# 5 Recommended Movies based on Searched Movies
dfRecommendedMovies.drop(searchedMovies)

,movieId,title,genres
6260,26913,Street Fighter II: The Animated Movie (Sutorît...,Action|Animation
8637,79274,Batman: Under the Red Hood (2010),Action|Animation
9570,99813,"Batman: The Dark Knight Returns, Part 2 (2013)",Action|Animation
10167,124867,Justice League: Throne of Atlantis (2015),Action|Animation
10277,138104,Justice League: Gods and Monsters (2015),Action|Animation


# Collaborative-Filtering
<hr>

In [43]:
ratingsPivot = ratings.pivot_table(index='userId',columns='movieId')
ratingsPivot

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2          5.0    NaN    2.0    NaN    3.0    NaN    NaN    NaN    NaN    NaN   
3          NaN    NaN    NaN    NaN    3.0    NaN    3.0    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...        ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
664        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
665        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
666        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
667        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
668        3.0    3.0    2.0    NaN    2.5    5.0    3.0    NaN    NaN    3.5   

         ...                                                                 \
movieId  ... 144482 144656 144976 146344 146656 146684 146878 148238 148626   
userId   ...                                                                  
1        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
664      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
665      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
666      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
667      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
668      ...    NaN    NaN    2.5    NaN    NaN    NaN    NaN    NaN    4.5   

                
movieId 149532  
userId          
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  
...        ...  
664        NaN  
665        NaN  
666        NaN  
667        NaN  
668        NaN  

[668 rows x 10325 columns]

In [44]:
ratingsPivot.replace(np.nan, 0, inplace=True)
ratingsPivot

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2          5.0    0.0    2.0    0.0    3.0    0.0    0.0    0.0    0.0    0.0   
3          0.0    0.0    0.0    0.0    3.0    0.0    3.0    0.0    0.0    0.0   
4          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5          4.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...        ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
664        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
665        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
666        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
667        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
668        3.0    3.0    2.0    0.0    2.5    5.0    3.0    0.0    0.0    3.5   

         ...                                                                 \
movieId  ... 144482 144656 144976 146344 146656 146684 146878 148238 148626   
userId   ...                                                                  
1        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
664      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
665      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
666      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
667      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
668      ...    0.0    0.0    2.5    0.0    0.0    0.0    0.0    0.0    4.5   

                
movieId 149532  
userId          
1          0.0  
2          0.0  
3          0.0  
4          0.0  
5          0.0  
...        ...  
664        0.0  
665        0.0  
666        0.0  
667        0.0  
668        0.0  

[668 rows x 10325 columns]

In [45]:
ratingsCorr = ratingsPivot.corr().loc['rating']
ratingsCorr.fillna(0, inplace=True)
ratingsCorr

rating                                                              \
movieId    1         2         3         4         5         6         7        
movieId                                                                         
1        1.000000  0.229124  0.222062  0.079319  0.243736  0.211601  0.237869   
2        0.229124  1.000000  0.108133  0.085610  0.123400  0.182489  0.111970   
3        0.222062  0.108133  1.000000  0.145779  0.469828  0.224401  0.281310   
4        0.079319  0.085610  0.145779  1.000000  0.095993  0.136973  0.067767   
5        0.243736  0.123400  0.469828  0.095993  1.000000  0.123640  0.366064   
...           ...       ...       ...       ...       ...       ...       ...   
146684  -0.027328 -0.015020 -0.011447 -0.004763 -0.011947 -0.017275 -0.011248   
146878  -0.027328 -0.015020 -0.011447 -0.004763 -0.011947 -0.017275 -0.011248   
148238  -0.027328  0.082551 -0.011447 -0.004763 -0.011947  0.105883  0.130140   
148626   0.026451  0.081235  0.029735 -0.008250  0.038672  0.109557  0.139296   
149532  -0.027328  0.082551 -0.011447 -0.004763 -0.011947  0.105883  0.130140   

                                       ...                                \
movieId    8         9         10      ...    144482    144656    144976   
movieId                                ...                                 
1        0.066370  0.026712  0.156136  ...  0.053224 -0.027328  0.054386   
2        0.035200  0.043333  0.314376  ...  0.082551  0.082551  0.035785   
3        0.161308  0.188836  0.039788  ... -0.011447 -0.011447  0.032102   
4        0.184446  0.006738  0.127035  ... -0.004763 -0.004763 -0.006649   
5        0.177667  0.086260  0.052069  ... -0.011947 -0.011947  0.040914   
...           ...       ...       ...  ...       ...       ...       ...   
146684  -0.002579 -0.006932 -0.019007  ... -0.001499 -0.001499 -0.002093   
146878  -0.002579 -0.006932 -0.019007  ... -0.001499 -0.001499 -0.002093   
148238  -0.002579 -0.006932  0.098554  ... -0.001499  1.000000 -0.002093   
148626  -0.004468  0.151573  0.134155  ... -0.002597 -0.002597  0.780272   
149532  -0.002579 -0.006932  0.098554  ... -0.001499  1.000000 -0.002093   

                                                                               
movieId    146344    146656    146684    146878    148238    148626    149532  
movieId                                                                        
1       -0.027328  0.026201 -0.027328 -0.027328 -0.027328  0.026451 -0.027328  
2        0.082551  0.115655 -0.015020 -0.015020  0.082551  0.081235  0.082551  
3       -0.011447 -0.016037 -0.011447 -0.011447 -0.011447  0.029735 -0.011447  
4       -0.004763 -0.006673 -0.004763 -0.004763 -0.004763 -0.008250 -0.004763  
5       -0.011947 -0.016738 -0.011947 -0.011947 -0.011947  0.038672 -0.011947  
...           ...       ...       ...       ...       ...       ...       ...  
146684  -0.001499 -0.002100  1.000000  1.000000 -0.001499 -0.002597 -0.001499  
146878  -0.001499 -0.002100  1.000000  1.000000 -0.001499 -0.002597 -0.001499  
148238   1.000000  0.599232 -0.001499 -0.001499  1.000000 -0.002597  1.000000  
148626  -0.002597 -0.003638 -0.002597 -0.002597 -0.002597  1.000000 -0.002597  
149532   1.000000  0.599232 -0.001499 -0.001499  1.000000 -0.002597  1.000000  

[10325 rows x 10325 columns]

In [148]:
# movieId of 'Being Flynn'
movies[movies['title'].str.contains('Being Flynn')]

,movieId,title,genres
9411,95816,Being Flynn (2012),Comedy|Drama


In [150]:
# Cause Widodo loved 'Being Flynn', let's say he gives it 5 star ratings
widodo = [(95816, 5)]

# Movie recommendation for Widodo
scoreSimilar = pd.DataFrame()
for movieId, ratingId in widodo:
    score = ratingsCorr.loc[movieId] * (ratingId)
    score = score.sort_values(ascending=False)
    scoreSimilar = scoreSimilar.append(score, ignore_index=True)
scoreSimilar

,"(rating, 7086)","(rating, 4801)","(rating, 6898)","(rating, 87660)","(rating, 53038)","(rating, 25930)","(rating, 41627)","(rating, 31309)","(rating, 40988)","(rating, 25898)",...,"(rating, 485)","(rating, 1485)","(rating, 19)","(rating, 551)","(rating, 2012)","(rating, 1080)","(rating, 1073)","(rating, 231)","(rating, 367)","(rating, 344)"
0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,-0.065876,-0.068753,-0.069383,-0.071272,-0.074308,-0.079633,-0.094189,-0.094907,-0.096955,-0.103537


In [141]:
# Top 5 recommendations based on score ratings
top5ratings = skorSimilar.iloc[:,:6]
top5ratings

,"(rating, 7086)","(rating, 4801)","(rating, 6898)","(rating, 87660)","(rating, 53038)","(rating, 25930)"
0,5.0,5.0,5.0,5.0,5.0,5.0


In [145]:
movieList = []
for i, movieId in top5ratings.columns:
    movieList.append(movieId)

# 5 movie recommendations for widodo
movies[movies['movieId'].isin(movieList)]

,movieId,title,genres
3755,4801,"Little Foxes, The (1941)",Drama
5020,6898,Sweet Sixteen (2002),Drama
5162,7086,Pygmalion (1938),Comedy|Drama
6000,25930,Odd Man Out (1947),Crime|Drama|Film-Noir|Thriller
7381,53038,Red Dust (1932),Drama
9033,87660,Too Big to Fail (2011),Drama
